In [14]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re
import html
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
%pip install xgboost
from xgboost import XGBClassifier

print("All libraries imported successfully!")
print("Environment setup complete!")



Note: you may need to restart the kernel to use updated packages.
All libraries imported successfully!
Environment setup complete!



### Business Understanding
A real world problem for any business is finding out the general sentiment about one's product. The data can obviously be found in twitter but a challenge is gathering the data and finding the average sentiment about a product. If one was to perform this task manually they can spend a large amount of time to discover what the general public thinks about their product. For a business this can be fatal since if a product has bad general sentiments the business will lose money since customers will stop buying the product and might influence others on not buying the product nor using the service. It is important for businesses to quickly know what the public thinks of their product, so that any needed changes can be made to salvage both the reputation of a business and to maximize the profits.

One way to solve this issue is by using a prediction model that can tell one if the sentiments about a product are positive or negative.The dataset we are using is a twitter dataset that was gathered by Crowdflower.

The first column is the text of the tweet, the second column denotes whether they are talking about google or apple and the third column states whether the sentiment is positive or negative.The main goal of the project is to predict the sentiment of a tweet from raw text.

The value of doing so is essential to any business because one can pull information from social media and get an overview of the sentiments about a certain product. This helps a company evaluate which of their products are the most popular. This allows the company decide which products they should invest more in. It can also tell a company which products are viewed unfavorably, so that the company can change the product. If a company is doing some A/B testing of a product through timed releases the sentiments online can be used to tell which version of a product is the best.

The business advantage of having reliable sentiment predictions is great. Shareholders as well as the workers in a company greatly benefit because of the reasons mentioned above.

The requirements for creating a model that can predict sentiments from tweets are:
        1.Raw Text from social media.
        2.Computational power to train the model.
        3.A team of data scientists to design the model.
        4.A well Labeled validation data to test the efficacy of a model.
'\'
The challenges of this project are:
        1.Gathering of textual data.
        2.Cleaning the raw text so that it is readable by a model.

The solution to the problems above:
        1.The CrowdFlower dataset gives one enough text scraped from twitter for one to perform the project.
        2.The NLTK library and regex python libraries provide one with the tools to clean textual data.
        3.The Sklearn python library will provide the models that will be trained on the cleaned textual data


###  Data Understanding

Before building any models, we took time to understand the structure and contents of the dataset. This step was essential to identify what kind of information we were working with and how to prepare it effectively.

1. Dataset Overview The dataset contains tweets that mention various brands or products. Along with each tweet, there's a label indicating the emotional response expressed toward the brand or product. These labels are useful for sentiment or emotion classification.

2. Main Columns The most important columns in the dataset are: tweet_text: The raw tweet content, which we use as the input feature,
Is there an emotion directed at a brand or product, The target label, which shows the type of emotional reaction (if any) present in the tweet.

3. Label Distribution We examined the unique classes in the target column to understand the types and balance of emotional responses. The Classes include:
-Positive emotion

-Negative emotion

-No emotion toward brand or product.

4.Missing Values We found some tweets with missing content. These rows were dropped during data preparation to ensure clean input.

5.Text Characteristics, we observed the following traits in the tweet text: Informal and noisy (containing emojis, hashtags, mentions, and URLs),
Varying lengths, from very short comments to longer sentences, Use of slang, repeated characters and abbreviations. This confirmed that proper text preprocessing (like cleaning, tokenizing, and lemmatization) would be necessary to improve model accuracy.



### Data Preparation
To get our dataset ready for modeling, we followed a structured set of data cleaning and preprocessing steps. Below is a summary of everything we did:

1. Data Loading. We began by loading the dataset, which contains tweets about products and companies. This gave us access to the raw text data along with labels indicating whether there was an emotion directed at a brand or product.

2.Initial Inspection. We examined the dataset to understand its structure. This included checking the number of rows and columns, identifying any missing values, and getting a general sense of the data types. We also looked at the unique values in the target column to understand the kinds of emotional labels we would be working with.

3.Stopword Setup. Before processing the text, we prepared a list of English stopwords. These are common words like “the”, “and”, “is”, which usually don’t add much value to analysis and can be removed.

4.Text Cleaning. Since our dataset was based on tweets, the text was informal and often messy. To make it usable for modeling, we applied several cleaning steps: 
-Emoji Removal: Emojis were removed to reduce noise.

-Hashtag and Mention Removal: We stripped out hashtags and user mentions (like @username) which were not relevant for our analysis.

-Link Removal: Any links or URLs in the tweets were removed.

-Special Character Filtering: We removed characters and symbols that were not letters, like punctuation or numbers.

-Whitespace Handling: Extra spaces were cleaned up to make the text consistent.

-Repeated Letters: Words with exaggerated letters (like “wooooow” or “heeeloooooo”) were cleaned by reducing excessive repetition.

5.Tokenization and Stopword Removal. We broke each tweet into individual words — a process called tokenization — and removed the stopwords we had prepared earlier. This helped us focus only on the most meaningful words in each tweet.

6.Testing and Validation to make sure our logic was working as expected, we tested some of the cleaning and filtering steps with simple examples. This helped us confirm that the functions we wrote were correctly removing or modifying the right parts of the text.

7.Pipeline Integration. We used  these steps into a pipeline using Scikit-learn’s tools. This made the process repeatable and easier to manage. The pipeline also included vectorization (using TF-IDF) and later integrated with various machine learning models.



In [15]:
#reading the data into a dataframe
df = pd.read_csv('judge-1377884607_tweet_product_company.csv',encoding='latin1')
df.head()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9093 entries, 0 to 9092
Data columns (total 3 columns):
 #   Column                                              Non-Null Count  Dtype 
---  ------                                              --------------  ----- 
 0   tweet_text                                          9092 non-null   object
 1   emotion_in_tweet_is_directed_at                     3291 non-null   object
 2   is_there_an_emotion_directed_at_a_brand_or_product  9093 non-null   object
dtypes: object(3)
memory usage: 213.2+ KB


In [17]:
df.describe()

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
count,9092,3291,9093
unique,9065,9,4
top,RT @mention Marissa Mayer: Google Will Connect...,iPad,No emotion toward brand or product
freq,5,946,5389


In [18]:
print((df.isnull().sum()/len(df))*100)

tweet_text                                             0.010997
emotion_in_tweet_is_directed_at                       63.807324
is_there_an_emotion_directed_at_a_brand_or_product     0.000000
dtype: float64


- Replaced null on the 'emotion_in_tweet_is_directed_at' column with 'Unknown' to keep more data for training and testing since it contains 64% of the dataset  
- Dropped for the remaining null values

In [19]:
df['emotion_in_tweet_is_directed_at'] = df['emotion_in_tweet_is_directed_at'].fillna('Unknown')

In [20]:
# Dropping null values for the remaining null values
df = df.dropna()

In [21]:
#looking at the classes in the target column
df['is_there_an_emotion_directed_at_a_brand_or_product'].unique()

array(['Negative emotion', 'Positive emotion',
       'No emotion toward brand or product', "I can't tell"], dtype=object)

Useful Columns in determining the sentiment of tweets include;

-Tweet_text
is_there_an_emotion_directed_at_a_brand_or_product
-The middle column which is called 'emotion_in_tweet_is_directed_at' is not useful for our aims. It will not be used in the training of the model.

We created a class named TweetPreprocessor and it will have functions for cleaning the text,Transforming the dataframe and creating a fit for training.

We will use this class within a pipeline. We will have our class inherit from Sklearn's BaseEstimator and TransformerMixin libraries.

BaseEstimator and TransformerMixin make a custom class:

1.Scikit-learn compliant.
2.Fully usable in Pipeline, GridSearchCV and cross_val_score.

In [22]:
class TweetPreprocessor(BaseEstimator, TransformerMixin):
    def __init__(self, 
                 remove_stopwords=True, 
                 min_token_len=3, #removing tokens with less than a length of 3
                 lemmatize_words=True,
                 output_format='tokens'):  # 'tokens' or 'string'
        
        self.remove_stopwords = remove_stopwords
        self.min_token_len = min_token_len
        self.lemmatize_words = lemmatize_words
        self.output_format = output_format

        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))

    def _is_english(self, text):
        try:
            return detect(text) == 'en'
        except:
            return False

    def clean_text(self, text):
        #to make sure that the text is in string format
        if not isinstance(text, str):
            text = str(text) if text is not None else ''

        # --- Preprocessing Steps ---
        text = emoji.replace_emoji(text, replace='')                     # Remove emojis
        text = text.lower()                                              # Lowercase
        text = re.sub(r'^rt\s+', '', text)                               # Remove 'RT' tag
        text = re.sub(r'https?://\S+', '', text)                         # Remove URLs
        text = re.sub(r'[@#\$]\w+', '', text)                            # Remove mentions, hashtags, cashtags
        text = re.sub(r'\d+', '', text)                                  # Remove numbers
        text = re.sub(r'(.)\1{2,}', r'\1', text)                         # Reduce character repetition
        text = re.sub(r'[^a-z\s]', '', text)                             # Keep only letters and spaces
        text = re.sub(r'\s+', ' ', text).strip()                         # Normalize whitespace

        # --- Tokenization ---
        tokens = word_tokenize(text)

        # --- Filtering ---
        #checking and removing stop words
        if self.remove_stopwords:
            tokens = [word for word in tokens if word not in self.stop_words]
        #checking for and removing tokens which are less than the threshold specified in init.
        if self.min_token_len > 0:
            tokens = [word for word in tokens if len(word) >= self.min_token_len]
        #lemmatizing
        if self.lemmatize_words:
            tokens = [self.lemmatizer.lemmatize(word) for word in tokens]

        # --- Return format ---
        #combining the tokens with spaces in the middle to create strings if we specify string in the output_format
        return ' '.join(tokens) if self.output_format == 'string' else tokens

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # Accept both list and Series (important for LIME compatibility)
        if isinstance(X, (list, pd.Series)):
            return [self.clean_text(text) for text in X]
        raise TypeError(f"Expected list or pd.Series, got {type(X)}")